In [25]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [26]:
print(os.listdir('C:\\Users\\Sairam Gundala\\Desktop\\class work\\kaggle\\malaria detection\\input'))

['cell_images']


In [27]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
data=[]
labels=[]
Parasitized=os.listdir("C:\\Users\\Sairam Gundala\\Desktop\\class work\\kaggle\\malaria detection\\input\\cell_images\\Parasitized")
for a in Parasitized:
    try:
        image=cv2.imread("C:\\Users\\Sairam Gundala\\Desktop\\class work\\kaggle\\malaria detection\\input\\cell_images\\Parasitized\\" + a)
        array=Image.fromarray(image, 'RGB')
        size_image=array.resize((50,50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print(" ")
Uninfected= os.listdir("C:\\Users\\Sairam Gundala\\Desktop\\class work\\kaggle\\malaria detection\\input\\cell_images\\Uninfected")
for a in Uninfected:
    try:
        image=cv2.imread("C:\\Users\\Sairam Gundala\\Desktop\\class work\\kaggle\\malaria detection\\input\\cell_images\\Uninfected\\" + a)
        array=Image.fromarray(image,'RGB')
        size_image=array.resize((50,50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print(" ")

In [31]:
Cells=np.array(data)
labels=np.array(labels)

In [32]:
n=np.arange(Cells.shape[0])
np.random.shuffle(n)

In [38]:
np.save('Cells',Cells)
np.save('labels',labels)

In [28]:
Cells=np.load("Cells.npy")
labels=np.load("labels.npy")

In [33]:
Cells=Cells[n]
labels=labels[n]

In [34]:
num_classes=len(np.unique(labels))
len_data=len(Cells)

In [35]:
(X_train,X_test)=Cells[int(0.1*len_data):],Cells[:int(0.1*len_data)]
X_train=X_train.astype('float32')/255
X_test=X_test.astype('float32')/255
train_length=len(X_train)
test_length=len(X_test)

In [36]:
(Y_train,Y_test)=labels[int(0.1*len_data):],labels[:int(0.1*len_data)]

In [37]:
Y_train=keras.utils.to_categorical(Y_train,num_classes)
Y_test=keras.utils.to_categorical(Y_test,num_classes)

In [38]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding='same',activation='relu',input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [40]:
model.fit(X_train,Y_train,batch_size=50,epochs=20,verbose=1)

Epoch 1/20
497/497 [==============================] - 23s 46ms/step - loss: 0.3637 - accuracy: 0.8266
Epoch 2/20
497/497 [==============================] - 23s 47ms/step - loss: 0.1571 - accuracy: 0.9505
Epoch 3/20
497/497 [==============================] - 27s 54ms/step - loss: 0.1426 - accuracy: 0.9558
Epoch 4/20
497/497 [==============================] - 28s 56ms/step - loss: 0.1318 - accuracy: 0.9584
Epoch 5/20
497/497 [==============================] - 28s 57ms/step - loss: 0.1244 - accuracy: 0.9596
Epoch 6/20
497/497 [==============================] - 29s 58ms/step - loss: 0.1162 - accuracy: 0.9610
Epoch 7/20
497/497 [==============================] - 27s 55ms/step - loss: 0.1092 - accuracy: 0.9622
Epoch 8/20
497/497 [==============================] - 26s 53ms/step - loss: 0.1018 - accuracy: 0.9646
Epoch 9/20
497/497 [==============================] - 24s 49ms/step - loss: 0.0974 - accuracy: 0.9653
Epoch 10/20
497/497 [==============================] - 24s 49ms/step - loss: 0.087

In [41]:
accuracy=model.evaluate(X_test,Y_test)
print(accuracy[1]*100)

87/87 [==============================] - 1s 7ms/step - loss: 0.1938 - accuracy: 0.9535
95.35390138626099


In [61]:
from keras.models import load_model

model.save('cell_predictions.h5')

In [6]:
model=load_model('UI//cell_predictions.h5')